In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import time
import pmdarima as pm

# Favorita

In [2]:
# Leitura dos dados tratados
favorita = pd.read_parquet('Bases Tratadas/favorita_item_loja_semana.parquet')
favorita['Data'] = pd.to_datetime(favorita['Data'])
display(favorita)

,Produto,Loja,Data,Vendas
0,96995,2,2013-01-07,2
33584,830624,15,2013-01-07,13
33585,830624,16,2013-01-07,52
33586,830624,17,2013-01-07,11
33587,830624,18,2013-01-07,4
...,...,...,...,...
23380695,903283,45,2017-08-14,377
23380696,903283,46,2017-08-14,60
23380697,903283,47,2017-08-14,114
23380687,903283,34,2017-08-14,113


In [3]:
# Agrupamento de vendas por data
favorita_agrupado = favorita[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
favorita_agrupado.set_index('Data', inplace=True)
display(favorita_agrupado)

,Vendas
Data,
2013-01-07,2546580
2013-01-14,2330352
2013-01-21,2384817
2013-01-28,2245352
2013-02-04,2534680
...,...
2017-07-17,5758675
2017-07-24,5738533
2017-07-31,5987120


In [4]:
# Dividir os dados
train_size = int(len(favorita_agrupado) * 0.7)
train, test = favorita_agrupado[:train_size], favorita_agrupado[train_size:]

# Criar e treinar o modelo AUTOARIMA com parâmetros otimizados
start_train_time = time.time()
auto_model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = auto_model.predict(n_periods=len(test))
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Garantir que os índices estão corretos
test_index = favorita_agrupado.index[-len(test):]
train_index = favorita_agrupado.index[:len(train)]

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=test.values.ravel(), mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict.ravel(), mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=train.values.ravel(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()

Tempo de treino: 59.325666427612305 segundos
Tempo de previsão: 0.01730799674987793 segundos


In [5]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado = pd.DataFrame({
    'Data': test_index,
    'Real': test.values.ravel(),
    'Previsto': test_predict.ravel()
})

# Salvar o dataframe em um arquivo CSV
df_resultado.to_csv('Resultados/Previsao/Favorita/Tudo/ARIMA.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo.to_csv('Resultados/Tempo/Favorita/Tudo/ARIMA.csv', index=False)

In [6]:
lojas_relevantes_favorita = [3, 8, 44, 45, 46, 47, 48, 49, 50, 51]
produtos_relevantes_favorita = {
    3: [1047679, 1503844],
    8: [1503844, 1047679],
    44: [1047679, 1503844],
    45: [1503844, 1473474],
    46: [584028, 1463992],
    47: [1503844, 1047679],
    48: [584028, 1463992],
    49: [1503844, 1473474],
    50: [584028, 364606],
    51: [1503844, 257847]
}

In [7]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_favorita])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_favorita):

    df_loja = favorita[favorita['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    df_loja_grouped.set_index('Data', inplace=True)
    
    train_size = int(len(df_loja_grouped) * 0.7)
    train, test = df_loja_grouped.iloc[:train_size], df_loja_grouped.iloc[train_size:]

    # Criar e treinar o modelo ARIMA
    start_train_time = time.time()
    model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time
    modelos_por_loja[loja] = model

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(n_periods=len(test))
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[train_size:]
    train_index = df_loja_grouped.index[:train_size]

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': test['Vendas'].values,
        'Previsto': test_predict
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=train['Vendas'], mode='lines', name=f'Treino Loja {loja}', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict, mode='lines', name=f'Previsto Loja {loja}', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test['Vendas'], mode='lines', name=f'Real Loja {loja}', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/Favorita/Loja/ARIMA.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Favorita/Loja/ARIMA.csv', index=False)

# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)


Tempos de treinamento por loja: {3: 60.10483169555664, 8: 48.86211800575256, 44: 92.9078917503357, 45: 60.79931879043579, 46: 286.6321258544922, 47: 734.6230428218842, 48: 217.00914525985718, 49: 52.894347190856934, 50: 243.07670783996582, 51: 54.55049705505371}
Tempos de previsão por loja: {3: 0.014029502868652344, 8: 0.014307975769042969, 44: 0.015110969543457031, 45: 0.015513896942138672, 46: 0.0170748233795166, 47: 0.020002365112304688, 48: 0.022001981735229492, 49: 0.01291656494140625, 50: 0.014004230499267578, 51: 0.01746964454650879}


In [8]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item = {}
tempos_treinamento = {}
tempos_previsao = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_favorita:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_favorita.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = favorita[(favorita['Loja'] == loja) & (favorita['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar vendas por data
        df_loja_item_grouped = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item_grouped.set_index('Data', inplace=True)

        # Dividir os dados em treino e teste
        train_size = int(len(df_loja_item_grouped) * 0.7)
        train, test = df_loja_item_grouped[:train_size], df_loja_item_grouped[train_size:]

        # Criar e treinar o modelo ARIMA
        start_train_time = time.time()
        model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
        model.fit(train)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time

        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = model.predict(n_periods=len(test))
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time

        # Garantir que os índices estão corretos
        test_index = test.index
        train_index = train.index

        # Adicionar resultados ao dataframe
        df_resultado_loja_item = pd.concat([df_resultado_loja_item, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': test['Vendas'].values,
            'Previsto': test_predict
        })])

        df_resultado_tempo_item = pd.concat([df_resultado_tempo_item, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })])

        # Plotando os dados originais
        fig_loja.add_trace(go.Scatter(x=test_index, y=test['Vendas'], mode='lines', name=f'Real Loja {loja} Item {item}', line=dict(color='orange')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict, mode='lines', name=f'Previsto Loja {loja} Item {item}', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=train_index, y=train['Vendas'], mode='lines', name=f'Treino Loja {loja} Item {item}', line=dict(color='blue')), row=idx+1, col=1)

        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

        # Salvar modelos e tempos
        modelos_por_loja_item[(loja, item)] = model
        tempos_treinamento[(loja, item)] = train_time
        tempos_previsao[(loja, item)] = forecast_time

    # Mostrar a figura para a loja atual
    fig_loja.update_layout(
        title=f'Previsão de Vendas para Loja {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    fig_loja.show()
    
    # Salvar os dataframes de resultados em arquivos CSV
df_resultado_loja_item.to_csv('Resultados/Previsao/Favorita/Item/ARIMA.csv', index=False)
df_resultado_tempo_item.to_csv('Resultados/Tempo/Favorita/Item/ARIMA.csv', index=False)


c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

# M5

In [9]:
file_path_m5 = 'Bases Tratadas/m5_item_loja_semana.parquet'
df_m5 = pd.read_parquet(file_path_m5)
display(df_m5.head())

,Produto,Loja,Data,Vendas
0,HOBBIES_1_001,CA_1,1,0
1,HOBBIES_1_002,CA_1,1,0
2,HOBBIES_1_003,CA_1,1,0
3,HOBBIES_1_004,CA_1,1,0
4,HOBBIES_1_005,CA_1,1,0


In [10]:
# Agrupamento de vendas por data
m5_agrupado = df_m5[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
m5_agrupado.set_index('Data', inplace=True)

display(m5_agrupado)

,Vendas
Data,
1,161932
2,194735
3,185293
4,170194
5,170777
...,...
273,287796
274,282731
275,314216


In [11]:
lojas_relevantes_m5 = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

produtos_relevantes_m5 = {
    'CA_1': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_2': ['FOODS_3_586', 'FOODS_3_252'],
    'CA_3': ['FOODS_3_090', 'FOODS_3_586'],
    'CA_4': ['FOODS_3_090', 'FOODS_3_586'],
    'TX_1': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_2': ['FOODS_3_586', 'FOODS_3_090'],
    'TX_3': ['FOODS_3_586', 'FOODS_3_090'],
    'WI_1': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_2': ['FOODS_3_226', 'FOODS_3_694'],
    'WI_3': ['FOODS_3_090', 'FOODS_3_226']
}

In [12]:
# Dividir os dados
train_size = int(len(m5_agrupado) * 0.7)
train, test = m5_agrupado[:train_size], m5_agrupado[train_size:]

# Criar e treinar o modelo AUTOARIMA com parâmetros otimizados
start_train_time = time.time()
auto_model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = auto_model.predict(n_periods=len(test))
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Garantir que os índices estão corretos
test_index = m5_agrupado.index[-len(test):]
train_index = m5_agrupado.index[:len(train)]

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=test.values.ravel(), mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict.ravel(), mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=train.values.ravel(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas - M5',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()

Tempo de treino: 664.7023868560791 segundos
Tempo de previsão: 0.024013757705688477 segundos


c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [13]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado_m5 = pd.DataFrame({
    'Data': test_index,
    'Real': test.values.ravel(),
    'Previsto': test_predict.ravel()
})

# Salvar o dataframe em um arquivo CSV
df_resultado_m5.to_csv('Resultados/Previsao/M5/Tudo/ARIMA.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo_m5 = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo_m5.to_csv('Resultados/Tempo/M5/Tudo/ARIMA.csv', index=False)

In [14]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_m5])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_m5):

    df_loja = df_m5[df_m5['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    df_loja_grouped.set_index('Data', inplace=True)
    
    train_size = int(len(df_loja_grouped) * 0.7)
    train, test = df_loja_grouped.iloc[:train_size], df_loja_grouped.iloc[train_size:]

    # Criar e treinar o modelo ARIMA
    start_train_time = time.time()
    model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time
    modelos_por_loja[loja] = model

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(n_periods=len(test))
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[train_size:]
    train_index = df_loja_grouped.index[:train_size]

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': test['Vendas'].values,
        'Previsto': test_predict
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=train['Vendas'], mode='lines', name=f'Treino Loja {loja}', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict, mode='lines', name=f'Previsto Loja {loja}', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test['Vendas'], mode='lines', name=f'Real Loja {loja}', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store - M5',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/M5/Loja/ARIMA.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/M5/Loja/ARIMA.csv', index=False)

# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

Tempos de treinamento por loja: {'CA_1': 538.684077501297, 'CA_2': 1772.7726361751556, 'CA_3': 850.8637380599976, 'CA_4': 1333.907497882843, 'TX_1': 954.9027388095856, 'TX_2': 337.6238844394684, 'TX_3': 910.3710787296295, 'WI_1': 34.559675455093384, 'WI_2': 841.4000079631805, 'WI_3': 499.1729989051819}
Tempos de previsão por loja: {'CA_1': 0.020654916763305664, 'CA_2': 0.07299923896789551, 'CA_3': 0.010518550872802734, 'CA_4': 0.06014132499694824, 'TX_1': 0.09372806549072266, 'TX_2': 0.009766101837158203, 'TX_3': 0.06828522682189941, 'WI_1': 0.005999088287353516, 'WI_2': 0.07209634780883789, 'WI_3': 0.010003805160522461}


In [15]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item_m5 = {}
tempos_treinamento_m5 = {}
tempos_previsao_m5 = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item_m5 = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item_m5 = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_m5:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_m5.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = df_m5[(df_m5['Loja'] == loja) & (df_m5['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar vendas por data
        df_loja_item_grouped = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
        df_loja_item_grouped.set_index('Data', inplace=True)

        # Dividir os dados em treino e teste
        train_size = int(len(df_loja_item_grouped) * 0.7)
        train, test = df_loja_item_grouped[:train_size], df_loja_item_grouped[train_size:]

        # Criar e treinar o modelo ARIMA
        start_train_time = time.time()
        model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
        model.fit(train)
        end_train_time = time.time()
        train_time = end_train_time - start_train_time

        # Realizar previsões no conjunto de teste
        start_forecast_time = time.time()
        test_predict = model.predict(n_periods=len(test))
        end_forecast_time = time.time()
        forecast_time = end_forecast_time - start_forecast_time

        # Garantir que os índices estão corretos
        test_index = test.index
        train_index = train.index

        # Adicionar resultados ao dataframe
        df_resultado_loja_item_m5 = pd.concat([df_resultado_loja_item_m5, pd.DataFrame({
            'Data': test_index,
            'Loja': loja,
            'Item': item,
            'Real': test['Vendas'].values,
            'Previsto': test_predict
        })])

        df_resultado_tempo_item_m5 = pd.concat([df_resultado_tempo_item_m5, pd.DataFrame({
            'Loja': [loja],
            'Item': [item],
            'Treino': [train_time],
            'Previsao': [forecast_time]
        })])

        # Plotando os dados originais
        fig_loja.add_trace(go.Scatter(x=test_index, y=test['Vendas'], mode='lines', name=f'Real Loja {loja} Item {item}', line=dict(color='orange')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict, mode='lines', name=f'Previsto Loja {loja} Item {item}', line=dict(color='green')), row=idx+1, col=1)
        fig_loja.add_trace(go.Scatter(x=train_index, y=train['Vendas'], mode='lines', name=f'Treino Loja {loja} Item {item}', line=dict(color='blue')), row=idx+1, col=1)

        fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
        fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

        # Salvar modelos e tempos
        modelos_por_loja_item_m5[(loja, item)] = model
        tempos_treinamento_m5[(loja, item)] = train_time
        tempos_previsao_m5[(loja, item)] = forecast_time

    # Mostrar a figura para a loja atual
    fig_loja.update_layout(
        title=f'Previsão de Vendas para Loja {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    fig_loja.show()
    
# Salvar os dataframes de resultados em arquivos CSV
df_resultado_loja_item_m5.to_csv('Resultados/Previsao/M5/Item/ARIMA.csv', index=False)
df_resultado_tempo_item_m5.to_csv('Resultados/Tempo/M5/Item/ARIMA.csv', index=False)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a 

# Olist

In [16]:
file_path_olist = 'Bases Tratadas/olist_item_loja_semana.parquet'
df_olist = pd.read_parquet(file_path_olist)
display(df_olist.head())

,Produto,Loja,Data,Vendas
0,c1488892604e4ba5cff5b4eb4d595400,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
1,f293394c72c9b5fafd7023301fc21fc2,1554a68530182680ad5c8b042c3ab563,2016-09-05,1
2,f3c2d01a84c947b078e32bbef0718962,a425f92c199eb576938df686728acd20,2016-09-05,1
3,5a6b04657a4c5ee34285d1e4619a96b4,ecccfa2bb93b34a3bf033cc5d1dcdc69,2016-09-19,3
4,fd7fd78fd3cbc1b0a6370a7909c0a629,f09b760d23495ac9a7e00d29b769007c,2016-10-03,1


In [17]:
# Agrupamento de vendas por data
olist_agrupado = df_olist[['Data', 'Vendas']].groupby('Data').sum().reset_index()

# Conversão de Data para o tipo datetime e definição como índice
olist_agrupado.set_index('Data', inplace=True)
display(olist_agrupado)

,Vendas
Data,
2016-09-05,3
2016-09-19,3
2016-10-03,9
2016-10-10,354
2016-12-26,1
...,...
2018-07-30,1865
2018-08-06,2423
2018-08-13,2145


In [18]:
lojas_relevantes_olist = [
    '6560211a19b47992c3666cc44a7e94c0', '4a3ca9315b744ce9f8e9374361493884', 
    '1f50f920176fa81dab994f9023523100', 'cc419e0650a3c5ba77189a1882b7556a', 
    'da8622b14eb17ae2831f4ac5b9dab84a', '955fee9216a65b617aa5c0531780ce60', 
    '1025f0e2d44d7041d6cf58b6550e0bfa', '7c67e1448b00f6e969d365cea6b010ab', 
    'ea8482cd71df3c1969d7b9473ff13abc', '7a67c85e85bb2ce8582c35f2203ad736'
]

produtos_relevantes_olist = {
    '6560211a19b47992c3666cc44a7e94c0': ['a92930c327948861c015c919a0bcb4a8', 'd017a2151d543a9885604dc62a3d9dcc'],
    '4a3ca9315b744ce9f8e9374361493884': ['99a4788cb24856965c36a24e339b6058', '35afc973633aaeb6b877ff57b2793310'],
    '1f50f920176fa81dab994f9023523100': ['422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b'],
    'cc419e0650a3c5ba77189a1882b7556a': ['154e7e31ebfa092203795c972e5804a6', '2b4609f8948be18874494203496bc318'],
    'da8622b14eb17ae2831f4ac5b9dab84a': ['e0cf79767c5b016251fe139915c59a26', 'fc1d8637c0268af3db482c14b7ef8e75'],
    '955fee9216a65b617aa5c0531780ce60': ['aca2eb7d00ea1a7b8ebd4e68314663af', '54d9ac713e253fa1fae9c8003b011c2a'],
    '1025f0e2d44d7041d6cf58b6550e0bfa': ['b532349fe46b38fbc7bb3914c1bdae07', '9ecadb84c81da840dbf3564378b586e9'],
    '7c67e1448b00f6e969d365cea6b010ab': ['8ed094bfe076c568f6bb10feada3f75d', '3eef0cb94ba82de806bb30ab743c7655'],
    'ea8482cd71df3c1969d7b9473ff13abc': ['97017430754804328eb9597b7f85da03', 'c1f5307decb89342351bec53668cffd9'],
    '7a67c85e85bb2ce8582c35f2203ad736': ['c6dd917a0be2a704582055949915ab32', '601a360bd2a916ecef0e88de72a6531a']
}

In [19]:
# Dividir os dados
train_size = int(len(olist_agrupado) * 0.7)
train, test = olist_agrupado[:train_size], olist_agrupado[train_size:]

# Criar e treinar o modelo AUTOARIMA com parâmetros otimizados
start_train_time = time.time()
auto_model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
end_train_time = time.time()
train_time = end_train_time - start_train_time

# Realizar previsões no conjunto de teste
start_forecast_time = time.time()
test_predict = auto_model.predict(n_periods=len(test))
end_forecast_time = time.time()
forecast_time = end_forecast_time - start_forecast_time

# Garantir que os índices estão corretos
test_index = olist_agrupado.index[-len(test):]
train_index = olist_agrupado.index[:len(train)]

print(f"Tempo de treino: {train_time} segundos")
print(f"Tempo de previsão: {forecast_time} segundos")

# Plotando os dados originais
fig = go.Figure()

# Add traces for real and predicted sales
fig.add_trace(go.Scatter(x=test_index, y=test.values.ravel(), mode='lines', name='Real', line=dict(color='orange')))
fig.add_trace(go.Scatter(x=test_index, y=test_predict.ravel(), mode='lines', name='Previsto', line=dict(color='green')))
fig.add_trace(go.Scatter(x=train_index, y=train.values.ravel(), mode='lines', name='Treino', line=dict(color='blue')))

# Add titles and labels
fig.update_layout(
    title='Vendas Reais vs Previstas - Olist',
    xaxis_title='Data',
    yaxis_title='Vendas',
    legend_title='Legenda',
    template='plotly_white'
)

# Show the figure
fig.show()

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (11) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

invalid value encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\_auto_solvers.py:38: RuntimeWarning:

invalid value encountered in divide

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppD

Tempo de treino: 32.12538409233093 segundos
Tempo de previsão: 0.008284807205200195 segundos


c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.



In [20]:
# Criar um dataframe com data, valor real e valor previsto
df_resultado_olist = pd.DataFrame({
    'Data': test_index,
    'Real': test.values.ravel(),
    'Previsto': test_predict.ravel()
})

# Salvar o dataframe em um arquivo CSV
df_resultado_olist.to_csv('Resultados/Previsao/Olist/Tudo/ARIMA.csv', index=False)

# Criar um dataframe com o tempo de treino e previsão
df_tempo_olist = pd.DataFrame({
    'Treino': [train_time],
    'Previsao': [forecast_time]
})
# Salvar o dataframe em um arquivo CSV
df_tempo_olist.to_csv('Resultados/Tempo/Olist/Tudo/ARIMA.csv', index=False)

In [21]:
modelos_por_loja = {}
tempos_treinamento = {}
tempos_previsao = {}

# Create a figure
fig = make_subplots(rows=5, cols=2, subplot_titles=[f'Loja {loja}' for loja in lojas_relevantes_olist])

df_resultado_loja = pd.DataFrame(columns=['Data', 'Loja', 'Real', 'Previsto'])
df_resultado_tempo = pd.DataFrame(columns=['Loja', 'Treino', 'Previsao'])

for i, loja in enumerate(lojas_relevantes_olist):

    df_loja = df_olist[df_olist['Loja'] == loja]
    df_loja_grouped = df_loja.groupby('Data')['Vendas'].sum().reset_index()

    date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
    date_range = date_range + pd.DateOffset(days=1)
    df_loja_grouped = df_loja_grouped.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
    df_loja_grouped.set_index('Data', inplace=True)
    
    train_size = int(len(df_loja_grouped) * 0.7)
    train, test = df_loja_grouped.iloc[:train_size], df_loja_grouped.iloc[train_size:]

    # Criar e treinar o modelo ARIMA
    start_train_time = time.time()
    model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
    end_train_time = time.time()
    train_time = end_train_time - start_train_time
    tempos_treinamento[loja] = train_time
    modelos_por_loja[loja] = model

    # Realizar previsões no conjunto de teste
    start_forecast_time = time.time()
    test_predict = model.predict(n_periods=len(test))
    end_forecast_time = time.time()
    forecast_time = end_forecast_time - start_forecast_time
    tempos_previsao[loja] = forecast_time

    # Garantir que os índices estão corretos
    test_index = df_loja_grouped.index[train_size:]
    train_index = df_loja_grouped.index[:train_size]

    df_resultado_loja = pd.concat([df_resultado_loja, pd.DataFrame({
        'Data': test_index,
        'Loja': loja,
        'Real': test['Vendas'].values,
        'Previsto': test_predict
    })])

    df_resultado_tempo = pd.concat([df_resultado_tempo, pd.DataFrame({
        'Loja': [loja],
        'Treino': [train_time],
        'Previsao': [forecast_time]
    })], ignore_index=True)

    # Add traces for train, test, and forecast
    fig.add_trace(go.Scatter(x=train_index, y=train['Vendas'], mode='lines', name=f'Treino Loja {loja}', line=dict(color='blue')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test_predict, mode='lines', name=f'Previsto Loja {loja}', line=dict(color='green')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter(x=test_index, y=test['Vendas'], mode='lines', name=f'Real Loja {loja}', line=dict(color='orange')), row=i//2+1, col=i%2+1)
    
    # Update layout for each subplot
    fig.update_xaxes(title_text='Data', row=i//2+1, col=i%2+1)
    fig.update_yaxes(title_text='Vendas', row=i//2+1, col=i%2+1)

# Update layout for the entire figure
fig.update_layout(
    title='Sales Forecast for Each Store - Olist',
    height=1500,
    showlegend=False,
    template='plotly_white'
)

# Salvar o dataframe em um arquivo CSV
df_resultado_loja.to_csv('Resultados/Previsao/Olist/Loja/ARIMA.csv', index=False)
df_resultado_tempo.to_csv('Resultados/Tempo/Olist/Loja/ARIMA.csv', index=False)

# Show the figure
fig.show()

# Print training and forecasting times
print("Tempos de treinamento por loja:", tempos_treinamento)
print("Tempos de previsão por loja:", tempos_previsao)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: Runtime

Tempos de treinamento por loja: {'6560211a19b47992c3666cc44a7e94c0': 26.370638132095337, '4a3ca9315b744ce9f8e9374361493884': 20.679890871047974, '1f50f920176fa81dab994f9023523100': 23.778340339660645, 'cc419e0650a3c5ba77189a1882b7556a': 28.080467700958252, 'da8622b14eb17ae2831f4ac5b9dab84a': 36.846312046051025, '955fee9216a65b617aa5c0531780ce60': 27.571196794509888, '1025f0e2d44d7041d6cf58b6550e0bfa': 8.22156047821045, '7c67e1448b00f6e969d365cea6b010ab': 6.100881576538086, 'ea8482cd71df3c1969d7b9473ff13abc': 34.237146854400635, '7a67c85e85bb2ce8582c35f2203ad736': 21.00882649421692}
Tempos de previsão por loja: {'6560211a19b47992c3666cc44a7e94c0': 0.012243032455444336, '4a3ca9315b744ce9f8e9374361493884': 0.010004997253417969, '1f50f920176fa81dab994f9023523100': 0.008019208908081055, 'cc419e0650a3c5ba77189a1882b7556a': 0.009241342544555664, 'da8622b14eb17ae2831f4ac5b9dab84a': 0.006999969482421875, '955fee9216a65b617aa5c0531780ce60': 0.008000612258911133, '1025f0e2d44d7041d6cf58b6550e0bfa

In [22]:
# Dicionários para armazenar os modelos e tempos para cada loja e item
modelos_por_loja_item_olist = {}
tempos_treinamento_olist = {}
tempos_previsao_olist = {}

# Criar dataframes para armazenar os resultados
df_resultado_loja_item_olist = pd.DataFrame(columns=['Data', 'Loja', 'Item', 'Real', 'Previsto'])
df_resultado_tempo_item_olist = pd.DataFrame(columns=['Loja', 'Item', 'Treino', 'Previsao'])

# Iterar sobre cada loja relevante
for loja in lojas_relevantes_olist:
    # Pegar os itens relevantes da loja atual
    itens_relevantes = produtos_relevantes_olist.get(loja, [])
    
    # Criar figura específica para a loja
    fig_loja = make_subplots(
        rows=len(itens_relevantes),
        cols=1,
        subplot_titles=[f'Loja {loja} - Item {item}' for item in itens_relevantes]
    )
    
    for idx, item in enumerate(itens_relevantes):
        df_loja_item = df_olist[(df_olist['Loja'] == loja) & (df_olist['Produto'] == item)]
        if df_loja_item.empty:
            continue  # Ignorar se não houver dados
        
        # Agrupar vendas por data
        try:
            df_loja_item_grouped = df_loja_item.groupby('Data')['Vendas'].sum().reset_index()
            date_range = pd.date_range(start=df_olist['Data'].min(), end=df_olist['Data'].max(), freq='W')
            date_range = date_range + pd.DateOffset(days=1)
            df_loja_item_grouped = df_loja_item_grouped.set_index('Data').reindex(date_range, fill_value=0).rename_axis('Data').reset_index()
            df_loja_item_grouped.set_index('Data', inplace=True)

            # Dividir os dados em treino e teste
            train_size = int(len(df_loja_item_grouped) * 0.7)
            train, test = df_loja_item_grouped[:train_size], df_loja_item_grouped[train_size:]

            # Criar e treinar o modelo ARIMA
            start_train_time = time.time()
            model = pm.auto_arima(train, seasonal=True, m=52, stepwise=True, error_action='ignore', suppress_warnings=True)
            model.fit(train)
            end_train_time = time.time()
            train_time = end_train_time - start_train_time

            # Realizar previsões no conjunto de teste
            start_forecast_time = time.time()
            test_predict = model.predict(n_periods=len(test))
            end_forecast_time = time.time()
            forecast_time = end_forecast_time - start_forecast_time

            # Garantir que os índices estão corretos
            test_index = test.index
            train_index = train.index

            # Adicionar resultados ao dataframe
            df_resultado_loja_item_olist = pd.concat([df_resultado_loja_item_olist, pd.DataFrame({
                'Data': test_index,
                'Loja': loja,
                'Item': item,
                'Real': test['Vendas'].values,
                'Previsto': test_predict
            })])

            df_resultado_tempo_item_olist = pd.concat([df_resultado_tempo_item_olist, pd.DataFrame({
                'Loja': [loja],
                'Item': [item],
                'Treino': [train_time],
                'Previsao': [forecast_time]
            })])

            # Plotando os dados originais
            fig_loja.add_trace(go.Scatter(x=test_index, y=test['Vendas'], mode='lines', name=f'Real Loja {loja} Item {item}', line=dict(color='orange')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=test_index, y=test_predict, mode='lines', name=f'Previsto Loja {loja} Item {item}', line=dict(color='green')), row=idx+1, col=1)
            fig_loja.add_trace(go.Scatter(x=train_index, y=train['Vendas'], mode='lines', name=f'Treino Loja {loja} Item {item}', line=dict(color='blue')), row=idx+1, col=1)

            fig_loja.update_xaxes(title_text='Data', row=idx+1, col=1)
            fig_loja.update_yaxes(title_text='Vendas', row=idx+1, col=1)

            # Salvar modelos e tempos
            modelos_por_loja_item_olist[(loja, item)] = model
            tempos_treinamento_olist[(loja, item)] = train_time
            tempos_previsao_olist[(loja, item)] = forecast_time
        except:
            print(f"Erro ao processar Loja {loja} Item {item}")

    # Mostrar a figura para a loja atual
    fig_loja.update_layout(
        title=f'Previsão de Vendas para Loja {loja}',
        height=300 * len(itens_relevantes),
        showlegend=False,
        template='plotly_white'
    )
    fig_loja.show()
    
# Salvar os dataframes de resultados em arquivos CSV
df_resultado_loja_item_olist.to_csv('Resultados/Previsao/Olist/Item/ARIMA.csv', index=False)
df_resultado_tempo_item_olist.to_csv('Resultados/Tempo/Olist/Item/ARIMA.csv', index=False)

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: Runtime

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAIN

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1899: RuntimeWarning:

divide by zero encountered in reciprocal

c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1906: RuntimeWarning:

divide by zero encountered in reciprocal



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1



c:\Users\BRAINER.CAMPOS\AppData\Local\anaconda3\envs\teste_tcc\lib\site-packages\pmdarima\arima\utils.py:117: UserWarning:

Appropriate D value may not have been reached; length of seasonally-differenced array (20) is shorter than m (52). Using D=1

